# Benchmarking Our Algorithm against others

In [1]:
import sys
sys.path.append('../../')
from time import time
import signal
from os import listdir
import numpy as np
import pickle as pkl


%load_ext autoreload
%autoreload 2

In [2]:
def get_env_name(filename):
    fs = filename.split('.')
    return fs[0]


def run_experiments(env_name, 
                    env_info, 
                    N_range, 
                    algs, 
                    TIMEOUT = 100,
                    results = [], 
                    executed = None):
    
    '''
        Checkpointed results and timeouted can be passed in so we can resume operation.
    '''

    if executed is None:
        executed = {alg: {N :False for N in N_range} for alg in algs}

   

    # Run the experiment on each algorithm
    for name,alg in algs.items():
        for N in N_range: 

            if executed[name][N]:
                continue
            
            # Construct the environment
            constructor = env_info['constructor']
            specs = env_info['specs']

            print('Building the environment...')
            env = constructor(**specs,N_agent = N)

            print("Env:", env_name, "Alg:", name, "N agent:",N)

            result = {}
            
            def timeout_handler(signum,frame):
                print("Algorithm timeout")
                # Mark the rest of the executed table as True, so we don't ever visit this algorithm again
                for n in executed[name]:
                    if n>=N:
                        executed[name][n] = True
                # Save checkpoint data
                with open("./data/{}.pkl".format(env_name),'wb') as fp:
                    pkl.dump({'results':results, 'executed':executed},fp)
                raise TimeoutError("Timeout")
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(TIMEOUT)
            try:
                t0 = time()
                plan = alg(env)
            except TimeoutError:
                print("Timeout")
            signal.alarm(0)

            t = time()-t0
            
            result['time'] = t

            result['N_agent'] = N
            result['alg_name'] = name

            result["TIMEOUT"]=TIMEOUT
            result["env"]=env

            results.append(result)

            # Mark the executed status as True
            executed[name][N] = True

            if t>TIMEOUT:
                print("Time out for algorithm", name, "N agent = ", N)
            else:
                result['plan'] = plan
                print("On time for algorithm", name, "N agent = ", N)

            # Save checkpoint data
            with open("./data/{}.pkl".format(env_name),'wb') as fp:
                pkl.dump({'results':results, 'executed':executed},fp)
            
def get_checkpoint_state(env_name,algs,N_range,first_run = False):
    if first_run:
        results = []
        executed = {alg: {N :False for N in N_range} for alg in algs}
        # Save checkpoint state
        with open("./data/{}.pkl".format(env_name),'wb') as fp:
            pkl.dump({'results':results, 'executed':executed},fp)

    # Load checkpoint state
    with open("./data/{}.pkl".format(env_name),'rb') as fp:
        ckpt = pkl.load(fp)

    results = ckpt['results']
    executed = ckpt['executed']

    return results,executed


In [3]:
from panav.sequential import sequential_HybridSIPP, sequential_planning
from panav.SAMP.solvers import Tube_Planning
from benchmarks.ST_RRTStar_wrapper import sequential_ST_RRTStar 
from panav.PIBT import PIBT_plan
from panav.PBS.PBS import PBS
from panav.TrafficAwarePlanning import TAHP


bloating_r = 0.5
vmax = 1.0

TIMEOUT = 200

# Specify the Algorithms
algs = {  
        "MILP": lambda HG: sequential_planning(Tube_Planning,HG.env,vmax,bloating_r,TIMEOUT=TIMEOUT), 
        "ST-RRT*": lambda HG: sequential_ST_RRTStar(HG.env,vmax,bloating_r),
        "PIBT": lambda HG: PIBT_plan(HG,vmax,bloating_r,TIMEOUT,simple_plan=False),
        "HybridSIPP": lambda HG: sequential_HybridSIPP(HG,return_on_failure=False)
        }
# Load environment presets
env_names = [get_env_name(fn) for fn in listdir('./envs')]

env_info = {}

for name in env_names:
    with open(f'./envs/{name}.pkl','rb') as fp:
        env_info[name] = pkl.load(fp)

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for ompl::msg::LogLevel already registered; second conversion method ignored.


In [4]:
env_name = 'Room'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed
N_range = np.arange(10,101,10)

In [5]:
results,executed = get_checkpoint_state(env_name,algs,N_range)
for N in executed['HybridSIPP']:
    executed['HybridSIPP'][N] = False
executed

{'MILP': {10: True,
  20: True,
  30: True,
  40: True,
  50: True,
  60: True,
  70: True,
  80: True,
  90: True,
  100: True},
 'ST-RRT*': {10: True,
  20: True,
  30: True,
  40: True,
  50: True,
  60: True,
  70: True,
  80: True,
  90: True,
  100: True},
 'PIBT': {10: True,
  20: True,
  30: True,
  40: True,
  50: True,
  60: True,
  70: True,
  80: True,
  90: True,
  100: True},
 'HybridSIPP': {10: False,
  20: False,
  30: False,
  40: False,
  50: False,
  60: False,
  70: False,
  80: False,
  90: False,
  100: False}}

In [6]:
# results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)
#
results,executed = get_checkpoint_state(env_name,algs,N_range)
for N in executed['HybridSIPP']:
    executed['HybridSIPP'][N] = False
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Building the environment...
Env: Room Alg: HybridSIPP N agent: 10
0
Km= 0
Solve time 0.08569192886352539
1
Km= 0
Solve time 0.0862431526184082
2
Km= 0
Solve time 0.09235668182373047
3
Km= 0
Solve time 0.08830070495605469
4
Km= 0
Solve time 0.08817791938781738
5
Km= 0
Solve time 0.08966302871704102
6
Km= 0
Solve time 0.08914303779602051
7
Km= 0
Solve time 0.0888528823852539
8
Km= 0
Solve time 0.28211021423339844
9
Km= 0
Solve time 0.09418797492980957
On time for algorithm HybridSIPP N agent =  10
Building the environment...
Env: Room Alg: HybridSIPP N agent: 20
0
Km= 0
Solve time 0.08748793601989746
1
Km= 0
Solve time 0.08988595008850098
2
Km= 0
Solve time 0.09450984001159668
3
Km= 0
Solve time 0.0894467830657959
4
Km= 0
Solve time 0.09201288223266602
5
Km= 0
Solve time 0.09300684928894043
6
Km= 0
Solve time 0.08880281448364258
7
Km= 0
Solve time 0.20753097534179688
8
Km= 0
Solve time 0.09997987747192383
9
Km= 0
Solve time 0.09241580963134766
10
Km= 0
Solve time 0.09260797500610352
11
K

/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1391: UserWarning: 
    The problem is either infeasible or unbounded, but the solver
    cannot tell which. Disable any solver-specific presolve methods
    and re-solve to determine the precise problem status.

    For GUROBI and CPLEX you can automatically perform this re-solve
    with the keyword argument prob.solve(reoptimize=True, ...).
    
  warnings.warn(INF_OR_UNB_MESSAGE)


Km= 2
Solve time 1.6634080410003662
On time for algorithm HybridSIPP N agent =  40
Building the environment...
Env: Room Alg: HybridSIPP N agent: 50
0
Km= 0
Solve time 0.17891883850097656
1
Km= 0
Solve time 0.09308791160583496
2
Km= 0
Solve time 0.08913207054138184
3
Km= 0
Solve time 0.08800721168518066
4
Km= 0
Solve time 0.0944981575012207
5
Km= 0
Solve time 0.08792495727539062
6
Km= 0
Solve time 0.09087991714477539
7
Km= 0
Solve time 0.09229779243469238
8
Km= 0
Solve time 0.0909261703491211
9
Km= 0
Solve time 0.09250402450561523
10
Km= 0
Solve time 0.0895853042602539
11
Km= 0
Solve time 0.21921896934509277
12
Km= 0
Solve time 0.09405994415283203
13
Km= 0
Solve time 0.09108519554138184
14
Km= 0
Solve time 0.0930318832397461
15
Km= 0
Solve time 0.09451103210449219
16
Km= 0
Solve time 0.0930180549621582
17
Km= 0
Solve time 0.09528017044067383
18
Km= 0
Solve time 0.09351968765258789
19
Km= 0
Solve time 0.09458804130554199
20
Km= 0
Solve time 0.09830808639526367
21
Km= 0
Solve time 0.1343

In [23]:
env_name = 'MultiTunnel'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed

N_range = np.arange(10,101,10)

In [ ]:
# results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)

In [33]:
results,executed = get_checkpoint_state(env_name,algs,N_range)
# rerun_alg = 'PIBT'
# results = [r for r in results if r['alg_name']!=rerun_alg]

# for N in executed[rerun_alg]:
#     executed[rerun_alg][N] = False

run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Building the environment...
Env: MultiTunnel Alg: PIBT N agent: 10
################# Time step 0 ################
Remaining agents 10
################# Time step 1 ################
Remaining agents 10
################# Time step 2 ################
Remaining agents 10
################# Time step 3 ################
Remaining agents 10
################# Time step 4 ################
Remaining agents 10
################# Time step 5 ################
Remaining agents 10
################# Time step 6 ################
Remaining agents 10
################# Time step 7 ################
Remaining agents 10
################# Time step 8 ################
Remaining agents 10
################# Time step 9 ################
Remaining agents 10
################# Time step 10 ################
Remaining agents 10
################# Time step 11 ################
Remaining agents 10
################# Time step 12 ################
Remaining agents 10
################# Time step 13 ################
Remaining a

/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1391: UserWarning: 
    The problem is either infeasible or unbounded, but the solver
    cannot tell which. Disable any solver-specific presolve methods
    and re-solve to determine the precise problem status.

    For GUROBI and CPLEX you can automatically perform this re-solve
    with the keyword argument prob.solve(reoptimize=True, ...).
    
  warnings.warn(INF_OR_UNB_MESSAGE)


################# Time step 73 ################
Remaining agents 20
################# Time step 74 ################
Remaining agents 20
################# Time step 75 ################
Remaining agents 20
################# Time step 76 ################
Remaining agents 20
################# Time step 77 ################
Remaining agents 20
################# Time step 78 ################
Remaining agents 20
################# Time step 79 ################
Remaining agents 20
################# Time step 80 ################
Remaining agents 20
################# Time step 81 ################
Remaining agents 20
################# Time step 82 ################
Remaining agents 20
################# Time step 83 ################
Remaining agents 20
################# Time step 84 ################
Remaining agents 20
################# Time step 85 ################
Remaining agents 20
################# Time step 86 ################
Remaining agents 20
################# Time step 87 ################


/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1385: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


################# Time step 219 ################
Remaining agents 14
################# Time step 220 ################
Remaining agents 14
################# Time step 221 ################
Remaining agents 14
################# Time step 222 ################
Remaining agents 14
################# Time step 223 ################
Remaining agents 14
################# Time step 224 ################
Remaining agents 14
################# Time step 225 ################
Remaining agents 14
################# Time step 226 ################
Remaining agents 14
################# Time step 227 ################
Remaining agents 14
################# Time step 228 ################
Remaining agents 14
################# Time step 229 ################
Remaining agents 14
################# Time step 230 ################
Remaining agents 14
################# Time step 231 ################
Remaining agents 14
################# Time step 232 ################
Remaining agents 14
################# Time step 233 ##

/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


################# Time step 186 ################
Remaining agents 69
Children 2 invalid
################# Time step 187 ################
Remaining agents 69
################# Time step 188 ################
Remaining agents 69
################# Time step 189 ################
Remaining agents 69
################# Time step 190 ################
Remaining agents 69
################# Time step 191 ################
Remaining agents 69
################# Time step 192 ################
Remaining agents 69
################# Time step 193 ################
Remaining agents 69
Children 48 invalid
################# Time step 194 ################
Remaining agents 69
Children 54 invalid
agent 24 reached
################# Time step 195 ################
Remaining agents 68
Children 48 invalid
Children 54 invalid
################# Time step 196 ################
Remaining agents 68
Children 48 invalid
################# Time step 197 ################
Remaining agents 68
################# Time step 198 ####

In [34]:
env_name = 'WareHouse'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed

N_range = np.arange(10,101,10)

In [35]:
results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)
# results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Building the environment...
Env: WareHouse Alg: MILP N agent: 10
Planning for agent 0/10


/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1391: UserWarning: 
    The problem is either infeasible or unbounded, but the solver
    cannot tell which. Disable any solver-specific presolve methods
    and re-solve to determine the precise problem status.

    For GUROBI and CPLEX you can automatically perform this re-solve
    with the keyword argument prob.solve(reoptimize=True, ...).
    
  warnings.warn(INF_OR_UNB_MESSAGE)


Planning for agent 1/10
Planning for agent 2/10
Planning for agent 3/10
Planning for agent 4/10
Planning for agent 5/10
Planning for agent 6/10
Planning for agent 7/10
Planning for agent 8/10
Planning for agent 9/10
On time for algorithm MILP N agent =  10
Building the environment...
Env: WareHouse Alg: MILP N agent: 20
Planning for agent 0/20
Planning for agent 1/20
Planning for agent 2/20
Planning for agent 3/20
Planning for agent 4/20
Planning for agent 5/20
Planning for agent 6/20
Planning for agent 7/20
Planning for agent 8/20
Planning for agent 9/20
Planning for agent 10/20
Planning for agent 11/20
Planning for agent 12/20
Planning for agent 13/20
Planning for agent 14/20
Planning for agent 15/20
Planning for agent 16/20
Planning for agent 17/20
Planning for agent 18/20
Planning for agent 19/20
On time for algorithm MILP N agent =  20
Building the environment...
Env: WareHouse Alg: MILP N agent: 30
Planning for agent 0/30
Planning for agent 1/30
Planning for agent 2/30
Planning f

         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Planning for agent 2/10Info:    SpaceTimeRRT: Created 18 states (7 start + 11 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Planning for agent 3/10
Info:    SpaceTimeRRT: Created 54 states (24 start + 30 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


...

Planning for agent 99/100Info:    SpaceTimeRRT: Created 44 states (16 start + 28 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


On time for algorithmInfo:    SpaceTimeRRT: Created 39 states (20 start + 19 goal)
 ST-RRT* N agent =  100
Building the environment...
Env: WareHouse Alg: PIBT N agent: 10
################# Time step 0 ################
Remaining agents 10
################# Time step 1 ################
Remaining agents 10
################# Time step 2 ################
Remaining agents 10
################# Time step 3 ################
Remaining agents 10
################# Time step 4 ################
Remaining agents 10
################# Time step 5 ################
Remaining agents 10
################# Time step 6 ################
Remaining agents 10
################# Time step 7 ################
Remaining agents 10
################# Time step 8 ################
Remaining agents 10
################# Time step 9 ################
Remaining agents 10
################# Time step 10 ################
Remaining agents 10
################# Time step 11 ################
Remaining agents 10
################# Time

/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1385: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


################# Time step 157 ################
Remaining agents 30
################# Time step 158 ################
Remaining agents 30
################# Time step 159 ################
Remaining agents 30
################# Time step 160 ################
Remaining agents 30
################# Time step 161 ################
Remaining agents 30
################# Time step 162 ################
Remaining agents 30
################# Time step 163 ################
Remaining agents 30
################# Time step 164 ################
Remaining agents 30
################# Time step 165 ################
Remaining agents 30
################# Time step 166 ################
Remaining agents 30
################# Time step 167 ################
Remaining agents 30
################# Time step 168 ################
Remaining agents 30
################# Time step 169 ################
Remaining agents 30
################# Time step 170 ################
Remaining agents 30
################# Time step 171 ##